In [50]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import glob

import pandas as pd

In [52]:
## delete the folders not in live config ( extra crap )
live_config_files = glob.glob('cfgs_live/*', recursive=True)
allowed_symbols = []
symbol_files = {}
for f in live_config_files:
    symbol = f.split('\\')[-1]
    symbol = symbol.replace('.json', 'USDT')
    allowed_symbols.append(symbol)
    symbol_files[symbol] = f
symbol_files
#allowed_symbols = ["ALICEUSDT","XRPUSDT","HOTUSDT"]

{'1000SHIBUSDT': 'cfgs_live\\1000SHIB.json',
 '1000XECUSDT': 'cfgs_live\\1000XEC.json',
 '1INCHUSDT': 'cfgs_live\\1INCH.json',
 'ADAUSDT': 'cfgs_live\\ADA.json',
 'AKROUSDT': 'cfgs_live\\AKRO.json',
 'ALGOUSDT': 'cfgs_live\\ALGO.json',
 'ALICEUSDT': 'cfgs_live\\ALICE.json',
 'ALPHAUSDT': 'cfgs_live\\ALPHA.json',
 'ANKRUSDT': 'cfgs_live\\ANKR.json',
 'API3USDT': 'cfgs_live\\API3.json',
 'ARUSDT': 'cfgs_live\\AR.json',
 'ATOMUSDT': 'cfgs_live\\ATOM.json',
 'BAKEUSDT': 'cfgs_live\\BAKE.json',
 'BANDUSDT': 'cfgs_live\\BAND.json',
 'BELUSDT': 'cfgs_live\\BEL.json',
 'BLZUSDT': 'cfgs_live\\BLZ.json',
 'BNBUSDT': 'cfgs_live\\BNB.json',
 'BTSUSDT': 'cfgs_live\\BTS.json',
 'C98USDT': 'cfgs_live\\C98.json',
 'CHZUSDT': 'cfgs_live\\CHZ.json',
 'COMPUSDT': 'cfgs_live\\COMP.json',
 'COTIUSDT': 'cfgs_live\\COTI.json',
 'CRVUSDT': 'cfgs_live\\CRV.json',
 'DENTUSDT': 'cfgs_live\\DENT.json',
 'DGBUSDT': 'cfgs_live\\DGB.json',
 'DODOUSDT': 'cfgs_live\\DODO.json',
 'DOGEUSDT': 'cfgs_live\\DOGE.json',
 'D

In [53]:
# ## NOW copy result folders :)
# import json
# import shutil
#
# files = glob.iglob('backtests/binance/' + '**/live_config.json', recursive=True)
#
# for filename in files:
#     with open(filename) as f:
#         config = json.load(f)
#     symbol = config['config_name'].split('_')[2]
#
#     if symbol not in symbol_files: continue
#
#     with open(symbol_files[symbol]) as f:
#         production_config = json.load(f)
#
#     long_pc = production_config['long']
#     long_c = config['long']
#
#     k_pc = long_pc.keys()
#     k_c = long_c.keys()
#
#     if len(k_c) != len(k_c): continue
#
#     same = True
#     for k in k_pc:
#         if k not in k_c:
#             same = False
#             break
#         v_pc = long_pc[k]
#         v_c = long_c[k]
#         #print(f'{k}={v_c},{v_pc}')
#
#     if not same: continue
#
#     print(f'{filename} is same as {symbol_files[symbol]}')
#
#     src = filename.replace('\\live_config.json', '')
#     dst = f'backtests/binance_production/{symbol}'
#     shutil.copytree(src=src, dst=dst, dirs_exist_ok=True)



In [54]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value
                if key == 'Production Config':
                    d['prod'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_605424/2885255594.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_605424/2885255594.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_605424/2885255594.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_605424/2885255594.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss,prod
0,\1000SHIBUSDT\plots\2022-04-03T194816\backtest...,5.5,1000SHIBUSDT,True,52.53,0.1780,0.066776,0.644511,100.0,0.8613,12.442375,True,-96.020,-1.88000,0.146280,0.802360,5.447,0.10870,155.190833,NaN
1,\1000SHIBUSDT\plots\2022-04-03T204145\backtest...,5.5,1000SHIBUSDT,True,26.73,0.0850,0.070106,0.830417,100.0,0.9153,6.572917,True,-95.820,-1.88000,0.148225,0.803949,5.404,0.09669,155.191667,NaN
2,\1000SHIBUSDT\plots\2022-04-06T180851\backtest...,5.5,1000SHIBUSDT,True,90.63,0.2460,0.065637,0.629854,100.0,0.7673,12.442375,True,-91.820,-1.64000,0.124514,0.747041,5.959,0.08869,180.079167,True
3,\1000XECUSDT\plots\2022-04-06T155612\backtest_...,5.5,1000XECUSDT,True,16.60,0.0401,0.056577,0.843518,100.0,0.8579,5.065292,True,49.790,0.20700,0.062401,0.371501,100.000,0.86990,21.804875,True
4,\1000XECUSDT\plots\2022-04-06T181135\backtest_...,5.5,1000XECUSDT,True,16.60,0.0401,0.056577,0.843518,100.0,0.8579,5.065292,True,49.790,0.20700,0.062401,0.371501,100.000,0.86990,21.804875,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,\ZECUSDT\plots\2022-04-04T105513\backtest_resu...,5.5,ZECUSDT,True,28.78,0.0950,0.075344,0.770541,100.0,0.9263,9.928458,True,18.230,0.02540,0.148103,0.660761,100.000,0.65240,114.903333,NaN
151,\ZECUSDT\plots\2022-04-07T123050\backtest_resu...,5.5,ZECUSDT,True,64.52,0.1770,0.074299,0.590802,100.0,0.8301,15.332625,True,-8.637,0.00538,0.138920,0.492966,88.210,0.65640,36.830542,True
152,\ZENUSDT\plots\2022-04-07T123822\backtest_resu...,5.5,ZENUSDT,True,71.30,0.2010,0.128784,0.968012,100.0,0.8159,14.339583,True,20.720,0.10100,0.109054,0.420619,49.720,0.56430,39.212500,True
153,\ZILUSDT\plots\2022-04-07T124653\backtest_resu...,5.5,ZILUSDT,True,11.86,0.0150,0.213304,1.171540,100.0,0.8399,35.809042,True,-90.810,-1.64000,0.128495,0.776683,5.828,0.07862,33.732625,True


Now filter the dataframe

In [55]:
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
                & (big_data['prod'] == 'True')
              # & (big_data['adgt_%'] > 0.1)
              # & (big_data['adgl_%'] > 0.04)
              # & (big_data['adgs_%'] > 0.04)
              # & (big_data['bnkrl_%'] > 99.0)
              # & (big_data['bnkrs_%'] > 99.0)
              # & (big_data['ebrs'] > .9)
              # #& (big_data['ebrl'] > .9)
              # & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,...,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss,prod,adgt_%
2,\1000SHIBUSDT\plots\2022-04-06T180851\backtest...,5.5,1000SHIBUSDT,True,90.63,0.2460,0.065637,0.629854,100.0,0.7673,...,True,-91.8200,-1.64000,0.124514,0.747041,5.959,0.08869,180.079167,True,-1.39400
3,\1000XECUSDT\plots\2022-04-06T155612\backtest_...,5.5,1000XECUSDT,True,16.60,0.0401,0.056577,0.843518,100.0,0.8579,...,True,49.7900,0.20700,0.062401,0.371501,100.000,0.86990,21.804875,True,0.24710
4,\1000XECUSDT\plots\2022-04-06T181135\backtest_...,5.5,1000XECUSDT,True,16.60,0.0401,0.056577,0.843518,100.0,0.8579,...,True,49.7900,0.20700,0.062401,0.371501,100.000,0.86990,21.804875,True,0.24710
5,\1INCHUSDT\plots\2022-04-06T182507\backtest_re...,5.5,1INCHUSDT,True,31.40,0.0995,0.122606,0.977967,100.0,0.8157,...,True,50.0100,0.14600,0.125987,0.618075,36.400,0.40490,33.669458,True,0.24550
8,\ADAUSDT\plots\2022-04-06T185658\backtest_resu...,5.5,ADAUSDT,True,26.33,0.0809,0.051500,0.391366,100.0,0.8072,...,True,36.5600,0.09610,0.101922,0.519449,75.970,0.64290,39.211792,True,0.17700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,\XRPUSDT\plots\2022-04-07T122119\backtest_resu...,5.5,XRPUSDT,True,38.19,0.1210,0.065995,0.830946,100.0,0.7749,...,True,0.7041,0.02980,0.126403,0.578940,76.630,0.57020,34.033333,True,0.15080
151,\ZECUSDT\plots\2022-04-07T123050\backtest_resu...,5.5,ZECUSDT,True,64.52,0.1770,0.074299,0.590802,100.0,0.8301,...,True,-8.6370,0.00538,0.138920,0.492966,88.210,0.65640,36.830542,True,0.18238
152,\ZENUSDT\plots\2022-04-07T123822\backtest_resu...,5.5,ZENUSDT,True,71.30,0.2010,0.128784,0.968012,100.0,0.8159,...,True,20.7200,0.10100,0.109054,0.420619,49.720,0.56430,39.212500,True,0.30200
153,\ZILUSDT\plots\2022-04-07T124653\backtest_resu...,5.5,ZILUSDT,True,11.86,0.0150,0.213304,1.171540,100.0,0.8399,...,True,-90.8100,-1.64000,0.128495,0.776683,5.828,0.07862,33.732625,True,-1.62500


In [56]:
df.drop_duplicates(
    subset=['s', 'prod', 'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
            'short'], keep='first',
    inplace=True)
dx = df[['file', 'prod', 's', 'adgt_%', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl','pastdl',
         'maxss', 'pamaxs','pastds' ]]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_605424/2052413415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(


,file,prod,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
2,\1000SHIBUSDT\plots\2022-04-06T180851\backtest...,True,1000SHIBUSDT,-1.39400,0.2460,-1.64000,100.0,5.959,0.7673,0.08869,12.442375,0.629854,0.065637,180.079167,0.747041,0.124514
3,\1000XECUSDT\plots\2022-04-06T155612\backtest_...,True,1000XECUSDT,0.24710,0.0401,0.20700,100.0,100.000,0.8579,0.86990,5.065292,0.843518,0.056577,21.804875,0.371501,0.062401
5,\1INCHUSDT\plots\2022-04-06T182507\backtest_re...,True,1INCHUSDT,0.24550,0.0995,0.14600,100.0,36.400,0.8157,0.40490,15.230542,0.977967,0.122606,33.669458,0.618075,0.125987
8,\ADAUSDT\plots\2022-04-06T185658\backtest_resu...,True,ADAUSDT,0.17700,0.0809,0.09610,100.0,75.970,0.8072,0.64290,6.515292,0.391366,0.051500,39.211792,0.519449,0.101922
9,\AKROUSDT\plots\2022-04-06T190136\backtest_res...,True,AKROUSDT,0.16250,0.1370,0.02550,100.0,100.000,0.8899,0.67510,16.145125,0.894140,0.142222,27.361792,0.502734,0.107376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,\XRPUSDT\plots\2022-04-07T122119\backtest_resu...,True,XRPUSDT,0.15080,0.1210,0.02980,100.0,76.630,0.7749,0.57020,15.477083,0.830946,0.065995,34.033333,0.578940,0.126403
151,\ZECUSDT\plots\2022-04-07T123050\backtest_resu...,True,ZECUSDT,0.18238,0.1770,0.00538,100.0,88.210,0.8301,0.65640,15.332625,0.590802,0.074299,36.830542,0.492966,0.138920
152,\ZENUSDT\plots\2022-04-07T123822\backtest_resu...,True,ZENUSDT,0.30200,0.2010,0.10100,100.0,49.720,0.8159,0.56430,14.339583,0.968012,0.128784,39.212500,0.420619,0.109054
153,\ZILUSDT\plots\2022-04-07T124653\backtest_resu...,True,ZILUSDT,-1.62500,0.0150,-1.64000,100.0,5.828,0.8399,0.07862,35.809042,1.171540,0.213304,33.732625,0.776683,0.128495


In [66]:
ds = dx[
   (dx['adgs_%'] > 0.00)
  #& (dx['bnkrs_%'] == 100.00)
]
ds = ds[['file', 'prod', 's', 'adgs_%', 'bnkrs_%','ebrs','maxss', 'pamaxs','pastds' ]]
ds

,file,prod,s,adgs_%,bnkrs_%,ebrs,maxss,pamaxs,pastds
3,\1000XECUSDT\plots\2022-04-06T155612\backtest_...,True,1000XECUSDT,0.20700,100.00,0.8699,21.804875,0.371501,0.062401
5,\1INCHUSDT\plots\2022-04-06T182507\backtest_re...,True,1INCHUSDT,0.14600,36.40,0.4049,33.669458,0.618075,0.125987
8,\ADAUSDT\plots\2022-04-06T185658\backtest_resu...,True,ADAUSDT,0.09610,75.97,0.6429,39.211792,0.519449,0.101922
9,\AKROUSDT\plots\2022-04-06T190136\backtest_res...,True,AKROUSDT,0.02550,100.00,0.6751,27.361792,0.502734,0.107376
10,\ALGOUSDT\plots\2022-04-06T191700\backtest_res...,True,ALGOUSDT,0.06150,80.31,0.6315,34.506250,0.473493,0.085239
...,...,...,...,...,...,...,...,...,...
142,\XMRUSDT\plots\2022-04-07T120315\backtest_resu...,True,XMRUSDT,0.02270,100.00,0.7453,48.270833,0.441080,0.113953
148,\XRPUSDT\plots\2022-04-07T122119\backtest_resu...,True,XRPUSDT,0.02980,76.63,0.5702,34.033333,0.578940,0.126403
151,\ZECUSDT\plots\2022-04-07T123050\backtest_resu...,True,ZECUSDT,0.00538,88.21,0.6564,36.830542,0.492966,0.138920
152,\ZENUSDT\plots\2022-04-07T123822\backtest_resu...,True,ZENUSDT,0.10100,49.72,0.5643,39.212500,0.420619,0.109054


In [69]:
short_symbols = ds['s'].tolist()
print(short_symbols)
config = pd.read_csv('run_config.csv')
config['usdt'] = config['symbol']+'USDT'
config = config[(config.usdt.isin(short_symbols))]
config

['1000XECUSDT', '1INCHUSDT', 'ADAUSDT', 'AKROUSDT', 'ALGOUSDT', 'ALICEUSDT', 'ANKRUSDT', 'API3USDT', 'ARUSDT', 'ATOMUSDT', 'BAKEUSDT', 'BELUSDT', 'BLZUSDT', 'BNBUSDT', 'BTSUSDT', 'C98USDT', 'CHZUSDT', 'COMPUSDT', 'DENTUSDT', 'DGBUSDT', 'DODOUSDT', 'DOGEUSDT', 'DOTUSDT', 'DUSKUSDT', 'DYDXUSDT', 'ENJUSDT', 'ETCUSDT', 'ETHUSDT', 'FLMUSDT', 'FLOWUSDT', 'GRTUSDT', 'HBARUSDT', 'HOTUSDT', 'ICPUSDT', 'ICXUSDT', 'IOSTUSDT', 'IOTAUSDT', 'KAVAUSDT', 'KLAYUSDT', 'KNCUSDT', 'LINKUSDT', 'LITUSDT', 'LTCUSDT', 'MATICUSDT', 'MKRUSDT', 'MTLUSDT', 'NEOUSDT', 'OGNUSDT', 'OMGUSDT', 'ONTUSDT', 'QTUMUSDT', 'RLCUSDT', 'ROSEUSDT', 'RVNUSDT', 'SCUSDT', 'SNXUSDT', 'STMXUSDT', 'SUSHIUSDT', 'SXPUSDT', 'TLMUSDT', 'TOMOUSDT', 'TRBUSDT', 'UNFIUSDT', 'VETUSDT', 'XLMUSDT', 'XMRUSDT', 'XRPUSDT', 'ZECUSDT', 'ZENUSDT', 'ZRXUSDT']


,Unnamed: 0,symbol,direction,long_mode,short_mode,long_exposure,short_exposure,usdt
0,0,ALICE,long,n,p,0.035,0.00,ALICEUSDT
1,1,IOST,long,n,p,0.035,0.00,IOSTUSDT
2,2,OMG,long,n,p,0.035,0.00,OMGUSDT
3,3,ATOM,long_short,n,n,0.035,0.02,ATOMUSDT
4,4,KAVA,long,n,p,0.035,0.00,KAVAUSDT
...,...,...,...,...,...,...,...,...
80,80,MATIC,long_short,n,n,0.035,0.02,MATICUSDT
81,81,LTC,long_short,n,n,0.035,0.02,LTCUSDT
82,82,LINK,long_short,n,n,0.035,0.02,LINKUSDT
83,83,GRT,long_short,n,n,0.035,0.02,GRTUSDT


In [62]:
## find the ones that are not supposed to be shorted!
short_symbols = ds['s'].tolist()

config = pd.read_csv('run_config.csv')
config['usdt'] = config['symbol']+'USDT'
config = config[(~config.usdt.isin(short_symbols)) & (config['short_mode'] == 'n')]
## find the risky ones!
not_supposed_to_be_shorted = config['usdt'].tolist()
not_supposed_to_be_shorted

['ATOMUSDT',
 'ALGOUSDT',
 'CHZUSDT',
 'ICPUSDT',
 'ETHUSDT',
 'ADAUSDT',
 'XMRUSDT',
 'MATICUSDT',
 'LTCUSDT']

In [64]:
dx
# ss = dx['s'].tolist()
# ss
ds_not_good = dx[(dx.s.isin(not_supposed_to_be_shorted))]
#s_not_good
#ds_not_good
ds_not_good

,file,prod,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
8,\ADAUSDT\plots\2022-04-06T185658\backtest_resu...,True,ADAUSDT,0.1770,0.0809,0.0961,100.0,75.97,0.8072,0.6429,6.515292,0.391366,0.051500,39.211792,0.519449,0.101922
10,\ALGOUSDT\plots\2022-04-06T191700\backtest_res...,True,ALGOUSDT,0.1855,0.1240,0.0615,100.0,80.31,0.8745,0.6315,22.408333,1.199200,0.146477,34.506250,0.473493,0.085239
22,\ATOMUSDT\plots\2022-04-06T202412\backtest_res...,True,ATOMUSDT,0.3030,0.1760,0.1270,100.0,64.62,0.9162,0.5639,16.588208,0.489647,0.081672,35.834708,0.577798,0.119612
36,\CHZUSDT\plots\2022-04-06T220927\backtest_resu...,True,CHZUSDT,0.2922,0.2110,0.0812,100.0,90.37,0.8257,0.6372,9.910417,0.755754,0.057503,27.648625,0.526384,0.110610
61,\ETHUSDT\plots\2022-04-07T030239\backtest_resu...,True,ETHUSDT,0.1846,0.1640,0.0206,100.0,100.00,0.7670,0.7239,37.025708,0.613138,0.086090,27.666667,0.451289,0.107923
76,\ICPUSDT\plots\2022-04-07T035310\backtest_resu...,True,ICPUSDT,0.2410,0.0930,0.1480,100.0,77.65,0.8746,0.5620,9.602083,0.797218,0.084918,27.669458,0.611777,0.126236
96,\LTCUSDT\plots\2022-04-07T053709\backtest_resu...,True,LTCUSDT,0.1578,0.1040,0.0538,100.0,96.92,0.8395,0.6471,7.831958,0.419116,0.051873,48.451250,0.530287,0.126367
101,\MATICUSDT\plots\2022-04-07T060521\backtest_re...,True,MATICUSDT,0.2241,0.1740,0.0501,100.0,93.79,0.8240,0.6528,27.195125,0.849123,0.132214,30.697917,0.500769,0.116346
142,\XMRUSDT\plots\2022-04-07T120315\backtest_resu...,True,XMRUSDT,0.1317,0.1090,0.0227,100.0,100.00,0.8318,0.7453,14.631958,0.477026,0.062808,48.270833,0.441080,0.113953
